In [ ]:
from pandas import concat, DataFrame
from requests import get, post


def _get_token(key: str) -> str:
    return post(
        url="https://apieco.eco-counter-tools.com/token",
        headers={"Authorization": f"Basic {key}"},
        data="grant_type=client_credentials",
    ).json()["access_token"]


def _get_raw_data(
    counter_id: int,
    start: str,
    stop: str,
    step: str,
    key: str,
) -> list[dict]:
    start, stop = [i.replace(":", "%3A") for i in [start, stop]]
    return get(
        url=f"https://apieco.eco-counter-tools.com/api/1.0/data/site/{counter_id}?begin={start}&end={stop}&step={step}",
        headers={
            "Accept": "application/json",
            "Authorization": f"Bearer {_get_token(key)}",
        },
    ).json()


def get_data(
    key: str,
    counter_name_id: dict[str, int],
    start: str = "2017-01-01T00:00:00",
    stop: str = "2022-01-01T00:00:00",
    step: str = "day",
) -> DataFrame:
    """Download counter data

    Args:
        key (str): Auth key (hash)
        counter_name_id (dict[str, int]): Dictionary of counter-names: ids
        start (str): Beginning of the period (ISO-8601 format: yyyy-mm-ddThh:mm:ss) (inclusive)
        stop (str): End of the period (ISO-8601 format: yyyy-mm-ddThh:mm:ss) (exclusive)
        step (str): Data step (allowed values: 15m/hour/day/week/month/year)

    Returns:
        DataFrame: With columns "Time" and counter names
    """
    data = [
        _get_raw_data(_id, start, stop, step, key) for _id in counter_name_id.values()
    ]

    return concat(
        [DataFrame(data[0]).date.rename("Time")]
        + [
            DataFrame(counter_data).counts.rename(counter_name)
            for counter_data, counter_name in zip(data, counter_name_id.keys())
        ],
        axis=1,
    )


counter_names_ids = {
    "Banks Lane": 100012747,
    "Blue Anchor Bay": 100012739,
    "Clevedon": 100030175,
    "Crickley Hill Country Park": 100012738,
    "Doniford Bridge": 100030174,
    "Fiddlers Island": 100012727,
    "Galley Hall": 100030170,
    "Holland Haven": 100030735,
    "Holme Dunes NNR": 100012736,
    "Itchenor": 100042765,
    "Malham Cove": 100012741,
    "Newlands Corner": 100012737,
    "Northey Island Causeway": 100030734,
    "Ogbourne St. George Post": 100012750,
    "Pegwell Bay": 100030736,
    "Rocket Post Field": 100012742,
    "Ryhope Dene": 100012735,
    "Saltburn": 100030177,
    "Sandwich Peninsula": 100030739,
    "Sandwich Peninsula  Bridge": 100046405,
    "Sennen Cove": 100012740,
    "Strete Gate": 100030737,
    "Teesdale Way": 100030176,
    "Trimingham": 100012725,
    "Undercliffs Natural Nature Reserve": 100030738,
    "Vessey Pastures": 100012743,
    "Walltown Quarry": 100012724,
    "Waxham": 100051237,
    "Wells-next-the-Sea NNR": 100055647,
    "Winterton Ness": 100051238,
}

In [ ]:
api_key = ""
path = "./data/strava_and_counter_data_updated/data_01-2020_to_11-2022/natural_england_national_trails_people_counters_daily.csv"

In [ ]:
get_data(
    api_key,
    counter_names_ids,
    start="2020-01-01T00:00:00",
    stop="2022-12-01T00:00:00",
    step="day",
).to_csv(
    path,
    index=False,
)